In [570]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbmfrom
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb
import catboost as catb
import lightgbm as lgbm
from sklearn.model_selection import GridSearchCV, cross_val_score, learning_curve
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from sklearn.base import clone
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from vecstack import stacking
from sklearn.ensemble import StackingClassifier

def make_predictions(filename, model):
    df_test = pd.read_csv('./data/test.csv')
    data_test = job(df_test)

    X_train_t, X_test_t, y_train_t, y_test_t, y_t, X_t, data_t, ids_t = get_data(False, data_test)

    y_predict =  model.predict(X_t)
    is_default = pd.DataFrame({'Id': ids_t,'Outcome':y_predict})

    is_default.to_csv(filename, index=False)

def get_classification_report(y_train_true, y_train_pred, y_test_true, y_test_pred):
    print('Train subset\n\n' + classification_report(y_train_true, y_train_pred))
    print('Test subset\n\n' + classification_report(y_test_true, y_test_pred))

def fit_and_show(clf, X_train, X_test, y_train, y_test):
    clf.fit(X_train, y_train)
    y_train_pred = clf.predict(X_train)
    y_test_pred = clf.predict(X_test)
    get_classification_report(y_train, y_train_pred, y_test, y_test_pred) 

def get_data(load, dt):
  
    if load:
        data = pd.read_pickle('work_data4.pkl')
    else:
        data = dt

    target = 'Outcome'  
    if 'Outcome' in data.columns:
        y = data[target]
    else:
        y = data['Id']
        data[target] = data['Id']        

    scaler = StandardScaler()
    ids = data['Id']

    X = scaler.fit_transform(data.drop([target, 'Id'], axis=1))
    
    X_train, X_test, y_train, y_test = train_test_split( X, y, shuffle=True, test_size=0.05, random_state=42)

    return X_train, X_test, y_train, y_test, y, X, data, ids


def correct_value(data, target, value):
    data.loc[data[target] < value, target] = data[target].median()
    return data

# генераторы признаков квадраты и логорифмы
def create_poly_features(data, num_features):
    poly_features = np.power(data[num_features], 2)
    poly_features = poly_features.rename(columns=dict(zip(data.columns, [col+' sqr' for col in data.columns])))
    return data.join(poly_features)

def create_log_features(data, num_features):
    log_features = np.log(data[num_features], where=data[num_features]>0)
    log_features = log_features.rename(columns=dict(zip(data.columns, [col+'_log' for col in data.columns])))
    return data.join(log_features)

# Подготовим функцию обработки дата сета
def job(data):
      # Заменим тех у кого не измерили глюкозу на средние значения
      data = correct_value(data, 'Glucose', 60)
      # Установим средние значения для кровянного давления, у кого не измерили
      data = correct_value(data, 'BloodPressure', 60)
      # Установим средние значения тех у кого индекс массы тела меньше 18
      data = correct_value(data,'BMI', 18)
      # откорректируем толщину кожы в 5 мм так как этот фактор влияет на усвоение инсулина, и его нужно было измерять, тут явно пропуски
      data = correct_value(data,'SkinThickness', 5)
      # Откорректируем нижнюю границу нормы по инсулину
      data = correct_value(data,'Insulin', 15)

      #data=data.drop(['BloodPressure', 'SkinThickness', 'Insulin'], axis=1)
      #data=data.drop(['Insulin'], axis=1)
      
      # Добавим логарифмы и квадраты для числовых переменных
      data = create_poly_features(data, num_features)
      data = create_log_features(data, num_features)
      return data
num_features = ['Pregnancies', 'Glucose', 'BMI', 'Age'] # Insulin

In [571]:
X_train, X_test, y_train, y_test, y, X, data, ids = get_data(True, [])

In [572]:
param_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : [0.001, 0.01, 0.015, 0.1],
    'solver' : ['liblinear', 'lgfbs', 'sag', 'saga'],
    'max_iter': [1000, 1500, 2000],
    'multi_class':['ovr', 'multinormal', 'auto']
    },

lg = LogisticRegression(random_state= 42)
lgs = GridSearchCV(lg, param_grid, n_jobs=-1, cv=5, scoring = 'f1')

fit_and_show(lgs, X_train, X_test, y_train, y_test)
lgs.best_params_

Train subset

              precision    recall  f1-score   support

           0       0.82      0.75      0.79       337
           1       0.61      0.70      0.65       187

    accuracy                           0.73       524
   macro avg       0.72      0.73      0.72       524
weighted avg       0.75      0.73      0.74       524

Test subset

              precision    recall  f1-score   support

           0       0.79      0.88      0.83        17
           1       0.78      0.64      0.70        11

    accuracy                           0.79        28
   macro avg       0.78      0.76      0.77        28
weighted avg       0.78      0.79      0.78        28



{'C': 0.001,
 'max_iter': 1000,
 'multi_class': 'ovr',
 'penalty': 'l2',
 'solver': 'liblinear'}

In [573]:
lr = LogisticRegression(
    solver='liblinear',
    C=0.001,
    max_iter= 1000,
    multi_class= 'ovr',
    penalty='l2',
      random_state=42)

In [574]:
lr2 = LogisticRegression(
     multi_class='auto',
     solver='liblinear',
      random_state=42)

In [575]:
lr3 = LogisticRegression(
solver='liblinear',
multi_class='auto',

max_iter=1000,
penalty='l2',
random_state=42)

In [576]:
rf = RandomForestClassifier(
                    max_depth=3,
                    min_samples_leaf=5,
                    criterion='entropy',
                    min_samples_split=5,
                    class_weight='balanced',
                    max_leaf_nodes=12,
                    random_state=43,
                    n_jobs=-1)

In [577]:
ada = AdaBoostClassifier(
    RandomForestClassifier(
        max_depth=2,
        min_samples_leaf=5,
        criterion='entropy',
        min_samples_split=5,
        class_weight='balanced',
        max_leaf_nodes=12,
        random_state=43,
        n_jobs=-1),
    
    algorithm='SAMME',
    random_state=43,
    learning_rate=0.11,
    n_estimators=27)


In [578]:
lgbt = LGBMClassifier(
        learning_rate=0.05,
        max_depth=2,
        metric='f1',
        n_estimators=90,
        random_state=42,
        reg_alpha=1,
        reg_lambda=0.5)

In [579]:
gb = GradientBoostingClassifier(
    max_depth=2,
    min_samples_leaf=5,
    min_samples_split=5,
    max_leaf_nodes=12,
    random_state=43,
        )

In [580]:
base_learners3 = [
    ('rf_1', ada),
    ]  

In [581]:
lrs = LogisticRegression(multi_class='auto', solver='liblinear', random_state=42)
sclf_st = StackingClassifier(estimators=base_learners3, final_estimator=lr3, cv=20)

In [582]:
fit_and_show(sclf_st, X_train, X_test, y_train, y_test)

Train subset

              precision    recall  f1-score   support

           0       0.87      0.86      0.86       337
           1       0.75      0.76      0.76       187

    accuracy                           0.83       524
   macro avg       0.81      0.81      0.81       524
weighted avg       0.83      0.83      0.83       524

Test subset

              precision    recall  f1-score   support

           0       0.85      1.00      0.92        17
           1       1.00      0.73      0.84        11

    accuracy                           0.89        28
   macro avg       0.93      0.86      0.88        28
weighted avg       0.91      0.89      0.89        28



In [583]:
make_predictions('predictions_stack2.csv', sclf_st)